In [14]:
import tensorflow as tf
import numpy as np

In [3]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [7]:
mnist.train.images.shape

(55000, 784)

In [8]:
28*28

784

In [9]:
mnist.train.labels

array([7, 3, 4, ..., 5, 6, 8], dtype=uint8)

In [12]:
np.asarray(mnist.train.labels, dtype=np.int32)

array([7, 3, 4, ..., 5, 6, 8], dtype=int32)

**tf.estimator.inputs.numpy_input_fn**(

    x,      #numpy array object or dict of numpy array objects. If an array, it will be treated as a single feature.
    y=None, #numpy array object or dict of numpy array object. None if absent.
    batch_size=128,
    num_epochs=1,
    shuffle=None,
    queue_capacity=1000,
    num_threads=1
)


In [4]:
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
shuffle=True)

In [28]:
train_input_fn()[0]["x"].shape

TensorShape([Dimension(100), Dimension(784)])

### The model function we'll use has the following call signature:

def **my_model_fn**(
    
       features, # This is batch_features from input_fn
       labels,   # This is batch_labels from input_fn
       mode,     # An instance of tf.estimator.ModeKeys
       params  # Additional configuration
):

In [43]:
tf.reset_default_graph()

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

def mnist_cnn(input):

    input_layer = tf.reshape(input, [-1, 28, 28, 1])

    print("input shape :", input_layer.shape)

    conv1 = tf.layers.conv2d(

          inputs=input_layer,
          filters=32,
          kernel_size=[5, 5],
          padding="same",
          activation=tf.nn.relu
    )

    print("conv1 shape :", conv1.shape)

    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    print("pool1 shape :", pool1.shape)

    conv2 = tf.layers.conv2d(
          inputs=pool1,
          filters=64,
          kernel_size=[5, 5],
          padding="same",
          activation=tf.nn.relu
    )

    print("conv2 shape :", conv2.shape)

    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    print("pool2 shape :", pool2.shape)

    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

    print("flatten shape :", pool2_flat.shape)
    
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    
    print("dense shape :", dense.shape)
    
    #To be used only in training mode
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training= True)
    
    print("dropout shape :", dropout.shape)
    
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    print("logit shape :", logits.shape)
    
    return logits
    
    
#features = train_input_fn()[0]
#logits = mnist_cnn(features["x"])
logits = mnist_cnn(tf.random_normal((100,784)))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(logits).shape)
    tf.summary.FileWriter('./log/',sess.graph)

input shape : (100, 28, 28, 1)
conv1 shape : (100, 28, 28, 32)
pool1 shape : (100, 14, 14, 32)
conv2 shape : (100, 14, 14, 64)
pool2 shape : (100, 7, 7, 64)
flatten shape : (100, 3136)
dense shape : (100, 1024)
dropout shape : (100, 1024)
logit shape : (100, 10)
(100, 10)


In [44]:
def cnn_model_fn(features, labels, mode):
    
      logits = mnist_cnn(features)
    
      predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
      }
    
      if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
      # Calculate Loss (for both TRAIN and EVAL modes)
      loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

      # Configure the Training Op (for TRAIN mode)
      if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

      # Add evaluation metrics (for EVAL mode)
      eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
      }
      return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

### tf.keras

In [33]:
Sequential = tf.keras.Sequential 
Conv2D = tf.keras.layers.Convolution2D
MaxPooling2D = tf.keras.layers.MaxPooling2D
Flatten = tf.keras.layers.Flatten
Dense = tf.keras.layers.Dense
Dropout = tf.keras.layers.Dropout

In [40]:
tf.reset_default_graph()
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(10, activation='softmax'))

In [36]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
__________

In [37]:
type(model)

tensorflow.python.keras._impl.keras.models.Sequential